Lokasi pengantaran  
Pulau A  
0 : Lokasi 0  
1 : Lokasi 1  
Pulau B  
2 : Lokasi 2  
3 : Lokasi 3  

Lokasi asal dan kembali  
4 : Lokasi 4  
5 : Lokasi 5  

Dua jenis kapal : 0 dan 1

In [1]:
import itertools
import math
import pandas as pd
import numpy as np

In [2]:
pulau_a = [0, 1]
pulau_b = [2, 3]
lokasi_asal = [4, 5]
demand_daily = [12, 23, 35, 44, 0, 0]
total_demand_ab = 307
each_storage = 500
fixed_capex = [1200000, 14500000, 11000000, 15000000, 3000000, 250000000]
data_kapal = [2500, 250000]
sewa_kapal = [2000, 20000]
konsumsi_kapal = [8,25]
harga_btu = 5

In [3]:
distances = {
    (0,0): 0,   (0,1): 694, (0,2): 951, (0,3): 641, (0,4): 651, (0,5): 111,
    (1,0): 694, (1,1): 0,   (1,2): 251, (1,3): 301, (1,4): 451, (1,5): 151,
    (2,0): 951, (2,1): 251, (2,2): 0,   (2,3): 691, (2,4): 331, (2,5): 141,
    (3,0): 641, (3,1): 301, (3,2): 691, (3,3): 0,   (3,4): 291, (3,5): 591,
    (4,0): 651, (4,1): 451, (4,2): 331, (4,3): 291, (4,4): 0,   (4,5): 211,
    (5,0): 111, (5,1): 151, (5,2): 141, (5,3): 591, (5,4): 211, (5,5): 0,
}

In [4]:
speed_str = input("Speed = ")
speed = int(speed_str)

Speed = 10


In [5]:
permts_ab = [l for l in itertools.permutations(range(4), 4)]
permts_a = [l for l in itertools.permutations(pulau_a, 2)]
permts_b = [l for l in itertools.permutations(pulau_b, 2)]

In [8]:
paths = []
path_distances = {}
path_times = {}
path_storage_sites = {}
path_storage_harga = {}
path_storages = {}
path_variable_capex = {}
path_total_capex = {}
results = []
kapal_path_konsumsi = {}
kapal_path_cost = {}
kapal_path_cost_yearly = {}
total_opex_yearly = {}
total_capex_opex = {}

for asal in lokasi_asal:
    for perm in permts_ab:
        #perm : (0, 1, 3, 2)
        #path : (4, 0, 1, 3, 2, 4)
        path = (asal,) + perm + (asal,)
        paths.append(path)
        path_distances[path] = 0
        path_times[path] = 0
        path_storage_sites[path] = []
        path_storage_harga[path] = []
        path_variable_capex[path] = 0
        path_total_capex[path] = 0
        
        for i in range(len(path)-1):
            edge = (path[i], path[i+1])
            path_distances[path] += distances[edge]
            path_times[path] += distances[edge] / speed + ((24+6)*3)
        
        for j in path:
            storage_site = math.ceil(path_times[path] * demand_daily[j] / 500)
            path_storage_sites[path].append(storage_site)
            
            variable_capex = storage_site * each_storage
            path_storage_harga[path].append(variable_capex)
            
            path_variable_capex[path] += variable_capex
            path_total_capex[path] += fixed_capex[j] + variable_capex

        #
        path_storages[path] = path_times[path] * total_demand_ab
        
        for k in range(len(konsumsi_kapal)):
            index = path + (konsumsi_kapal[k],)
            kapal_path_konsumsi[index] = path_times[path] * konsumsi_kapal[k]
            kapal_path_cost[index] = kapal_path_konsumsi[index] * harga_btu
            kapal_path_cost_yearly[index] = kapal_path_cost[index] * 365 / path_times[path]

            if data_kapal[k] <= path_storages[path]:
                row = [path, k, kapal_path_konsumsi[index], kapal_path_cost[index], kapal_path_cost_yearly[index], 0, 0, "Not OK"]
                results.append(row)
            else :
                biaya_sewa = sewa_kapal[k] * 365
                total_opex_yearly[index] = kapal_path_cost_yearly[index] + biaya_sewa
                total_capex_opex[index] = total_opex_yearly[index] + path_total_capex[path]
                
                row = [path, k, kapal_path_konsumsi[index], kapal_path_cost[index], kapal_path_cost_yearly[index], total_opex_yearly[index], total_capex_opex[index], "OK"]
                results.append(row)

In [9]:
A = np.array(results, dtype=object)

In [17]:
hasil = {'Rute': A[:,0], 'Kapal': A[:,1], 'Konsumsi': A[:,2], 'Cost': A[:,3], 'Cost Yearly': A[:,4], 'Opex Yearly': A[:,5], 'Capex Opex': A[:,6], 'OK': A[:,7]}
df = pd.DataFrame(hasil, columns = ['Rute', 'Kapal', 'Konsumsi', 'Cost', 'Cost Yearly', 'Opex Yearly', 'Capex Opex', 'OK'])
df

,Rute,Kapal,Konsumsi,Cost,Cost Yearly,Opex Yearly,Capex Opex,OK
0,"(4, 0, 1, 2, 3, 4)",0,5662.4,28312,14600,0,0,Not OK
1,"(4, 0, 1, 2, 3, 4)",1,17695,88475,45625,7.34562e+06,5.51271e+07,OK
2,"(4, 0, 1, 3, 2, 4)",0,5734.4,28672,14600,0,0,Not OK
3,"(4, 0, 1, 3, 2, 4)",1,17920,89600,45625,7.34562e+06,5.51286e+07,OK
4,"(4, 0, 2, 1, 3, 4)",0,5556,27780,14600,0,0,Not OK
...,...,...,...,...,...,...,...,...
91,"(5, 3, 1, 2, 0, 5)",1,16762.5,83812.5,45625,7.34562e+06,5.49123e+08,OK
92,"(5, 3, 2, 0, 1, 5)",0,6062.4,30312,14600,0,0,Not OK
93,"(5, 3, 2, 0, 1, 5)",1,18945,94725,45625,7.34562e+06,5.49133e+08,OK
94,"(5, 3, 2, 1, 0, 5)",0,5470.4,27352,14600,0,0,Not OK


In [15]:
df.to_excel ('export_dataframe.xlsx', index = False, header=True)